In [7]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [20]:
train_data = pd.read_csv("../train.csv")
train_data = train_data.dropna(subset=['gt_25k_p6', 'md_earn_wne_p10'])

In [37]:
train_data.columns

Index(['INSTNM', 'CITY', 'STABBR', 'PREDDEG', 'CONTROL', 'LOCALE', 'SATVRMID',
       'SATMTMID', 'SATWRMID', 'ACTCMMID', 'ACTENMID', 'ACTMTMID', 'ACTWRMID',
       'SAT_AVG', 'DISTANCEONLY', 'UGDS', 'UGDS_WHITE', 'UGDS_BLACK',
       'UGDS_HISP', 'UGDS_ASIAN', 'UGDS_AIAN', 'UGDS_NHPI', 'UGDS_2MOR',
       'UGDS_NRA', 'UGDS_UNKN', 'PPTUG_EF', 'NPT4_PUB', 'NPT4_PRIV', 'PCTPELL',
       'RET_FT4', 'RET_FTL4', 'RET_PT4', 'RET_PTL4', 'PCTFLOAN', 'UG25abv',
       'GRAD_DEBT_MDN_SUPP', 'GRAD_DEBT_MDN10YR_SUPP', 'RPY_3YR_RT_SUPP',
       'C150_4_POOLED_SUPP', 'C200_L4_POOLED_SUPP', 'md_earn_wne_p10',
       'gt_25k_p6', 'gt_25k_05_p6'],
      dtype='object')

# 1

## a

In [30]:
md_earn_mean = train_data["md_earn_wne_p10"].mean()
gt_25k_mean = train_data["gt_25k_05_p6"].mean()
gt_25k_mode = gt_25k_mean >= .5

In [31]:
gt_25k_mode

True

In [32]:
md_earn_MSE = np.mean([(v - md_earn_mean)**2 for v in train_data["md_earn_wne_p10"].values])
gt_25k_MSE = np.mean([(v - gt_25k_mode)**2 for v in train_data["gt_25k_05_p6"].values])

In [34]:
md_earn_MSE

191640717.99292535

In [35]:
gt_25k_MSE

0.4686660698299015

## b

It's the right loss function. We're just trying to get them right.

## c

In [55]:
for col in X_train.columns:
    r = X_train[col].isna().sum()
    print(col, r)

INSTNM 0
CITY 0
STABBR 0
PREDDEG 0
CONTROL 0
LOCALE 157
SATVRMID 2611
SATMTMID 2604
SATWRMID 2888
ACTCMMID 2579
ACTENMID 2680
ACTMTMID 2680
ACTWRMID 3166
SAT_AVG 2543
DISTANCEONLY 157
UGDS 186
UGDS_WHITE 186
UGDS_BLACK 186
UGDS_HISP 186
UGDS_ASIAN 186
UGDS_AIAN 186
UGDS_NHPI 186
UGDS_2MOR 186
UGDS_NRA 186
UGDS_UNKN 186
PPTUG_EF 187
NPT4_PUB 2315
NPT4_PRIV 1325
PCTPELL 187
RET_FT4 2120
RET_FTL4 1717
RET_PT4 2540
RET_PTL4 2325
PCTFLOAN 187
UG25abv 193
GRAD_DEBT_MDN_SUPP 326
GRAD_DEBT_MDN10YR_SUPP 326
RPY_3YR_RT_SUPP 287
C150_4_POOLED_SUPP 2049
C200_L4_POOLED_SUPP 1733
md_earn_wne_p10 0
gt_25k_p6 0
gt_25k_05_p6 0


In [64]:
predictors = ['CONTROL', 'LOCALE', 'UGDS', 'PCTPELL', 'PCTFLOAN']
train_data = train_data.dropna(subset=predictors)

In [65]:
from sklearn.model_selection import train_test_split
X_train, X_test, _ ,_ = train_test_split(train_data, train_data, test_size=0.25)

In [68]:
ols = smf.ols('md_earn_wne_p10 ~ C(CONTROL) + C(LOCALE) + UGDS + PCTPELL + PCTFLOAN', data=X_train).fit()

In [69]:
ols.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        md_earn_wne_p10   R-squared:                       0.440
Model:                            OLS   Adj. R-squared:                  0.437
Method:                 Least Squares   F-statistic:                     154.9
Date:                Wed, 24 Apr 2019   Prob (F-statistic):               0.00
Time:                        21:35:05   Log-Likelihood:                -33338.
No. Observations:                3170   AIC:                         6.671e+04
Df Residuals:                    3153   BIC:                         6.681e+04
Df Model:                          16                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept          4.055e+04    708.200     57.251      0.000    3.92e+04    4.19e+04
C(CONTROL)[T.2]    2058.0925    521.930      3.943      0.000    1034.735    3081.450
C(CONTROL)[T.3]   -4993.5559    558.948     -8.934      0.000   -6089.494   -3897.617
C(LOCALE)[T.12.0] -1692.7845    566.693     -2.987      0.003   -2803.909    -581.660
C(LOCALE)[T.13.0] -2935.7234    565.926     -5.187      0.000   -4045.345   -1826.102
C(LOCALE)[T.21.0] -1194.8713    467.214     -2.557      0.011   -2110.946    -278.797
C(LOCALE)[T.22.0] -3221.6315   1031.148     -3.124      0.002   -5243.420   -1199.843
C(LOCALE)[T.23.0] -3121.0909   1299.318     -2.402      0.016   -5668.685    -573.497
C(LOCALE)[T.31.0] -5363.5872   1148.953     -4.668      0.000   -7616.358   -3110.817
C(LOCALE)[T.32.0] -5073.1143    743.390     -6.824      0.000   -6530.692   -3615.537
C(LOCALE)[T.33.0] -4788.4262    776.558     -6.166      0.000   -6311.036   -3265.816
C(LOCALE)[T.41.0] -3785.2740    789.244     -4.796      0.000   -5332.759   -2237.789
C(LOCALE)[T.42.0] -6004.2874   1325.865     -4.529      0.000   -8603.933   -3404.642
C(LOCALE)[T.43.0] -5783.0672   1826.934     -3.165      0.002   -9365.167   -2200.967
UGDS                  0.2350      0.035      6.628      0.000       0.165       0.305
PCTPELL           -2.815e+04    949.423    -29.650      0.000      -3e+04   -2.63e+04
PCTFLOAN           1.808e+04    728.734     24.814      0.000    1.67e+04    1.95e+04
==============================================================================
Omnibus:                     1043.922   Durbin-Watson:                   2.036
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             5279.357
Skew:                           1.490   Prob(JB):                         0.00
Kurtosis:                       8.576   Cond. No.                     7.53e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.53e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [70]:
log = smf.logit('gt_25k_05_p6 ~ C(CONTROL) + C(LOCALE) + UGDS + PCTPELL + PCTFLOAN', data=X_train).fit()

Optimization terminated successfully.
         Current function value: 0.464923
         Iterations 7


In [71]:
log.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           gt_25k_05_p6   No. Observations:                 3170
Model:                          Logit   Df Residuals:                     3153
Method:                           MLE   Df Model:                           16
Date:                Wed, 24 Apr 2019   Pseudo R-squ.:                  0.3285
Time:                        21:35:18   Log-Likelihood:                -1473.8
converged:                       True   LL-Null:                       -2194.7
                                        LLR p-value:                1.699e-297
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             1.1371      0.205      5.559      0.000       0.736       1.538
C(CONTROL)[T.2]      -0.5436      0.172     -3.161      0.002      -0.881      -0.207
C(CONTROL)[T.3]      -1.3427      0.167     -8.040      0.000      -1.670      -1.015
C(LOCALE)[T.12.0]    -0.1547      0.163     -0.952      0.341      -0.473       0.164
C(LOCALE)[T.13.0]    -0.3726      0.167     -2.233      0.026      -0.700      -0.046
C(LOCALE)[T.21.0]     0.0466      0.130      0.358      0.720      -0.208       0.302
C(LOCALE)[T.22.0]    -0.4129      0.299     -1.380      0.168      -0.999       0.174
C(LOCALE)[T.23.0]    -0.3305      0.397     -0.832      0.405      -1.109       0.448
C(LOCALE)[T.31.0]    -0.4742      0.346     -1.372      0.170      -1.152       0.203
C(LOCALE)[T.32.0]    -0.5163      0.224     -2.302      0.021      -0.956      -0.077
C(LOCALE)[T.33.0]    -0.4883      0.222     -2.196      0.028      -0.924      -0.052
C(LOCALE)[T.41.0]    -0.2133      0.220     -0.971      0.331      -0.644       0.217
C(LOCALE)[T.42.0]    -1.3423      0.407     -3.297      0.001      -2.140      -0.544
C(LOCALE)[T.43.0]    -0.7263      0.594     -1.222      0.222      -1.891       0.438
UGDS               6.581e-05   1.22e-05      5.389      0.000    4.19e-05    8.97e-05
PCTPELL              -8.0034      0.383    -20.901      0.000      -8.754      -7.253
PCTFLOAN              6.9504      0.329     21.116      0.000       6.305       7.596
=====================================================================================
"""

In [75]:
def MSE(preds, labels):
    return np.mean([(v - p)**2 for v,p in zip(labels, preds)])

In [78]:
ols_pred = ols.predict(X_test)
log_pred = log.predict(X_test)
ols_MSE = MSE(X_test["md_earn_wne_p10"].values,ols_pred)
log_MSE = MSE(X_test["gt_25k_05_p6"].values,log_pred)

In [79]:
ols_MSE

70210619.39437677

In [80]:
log_MSE

0.1523019079886964

MUCH BETTER!!!!!

# 2